# Assignment 2

In [25]:
import threading
import time
import random
from enum import Enum
from pynq.overlays.base import BaseOverlay
import asyncio
import pynq.lib.rgbled as rgbled
base = BaseOverlay("base.bit")

In [28]:
# Asyncio control loop flag
cond = True

# Board LEDs
board_leds = [
    base.leds[i] for i in range(4)
]
board_leds.append(rgbled.RGBLED(4))

# Atomic fork pickup flag
fork_pickup = asyncio.Lock()

# Range of napping and starving times (for randint)
NAPPING_STARVING_MAX = 8
NAPPING_STARVING_MIN = 2

# Defining states for state machine
class PhilosopherState(Enum):
    STARVING = 1
    EATING   = 2
    NAPPING  = 3

# asyncio lock wrapper
class Fork:
    def __init__(self, idx):
        self.lock  = threading.Lock()
        self.idx   = idx
    def locked(self):
        return self.lock.locked()
    def acquire(self,blocking=True):
        return self.lock.acquire(blocking)
    def release(self):
        return self.lock.release()

class Philosopher:
    def __init__(self, left_fork: Fork, right_fork: Fork, led, state=PhilosopherState.STARVING):
        self.state = state
        self.left_fork = left_fork
        self.right_fork = right_fork
        self.led = led
        self.state_condition = asyncio.Condition()      # for notifying watchers
        self.eat_time = random.randint(NAPPING_STARVING_MIN, NAPPING_STARVING_MAX-1)
        self.nap_time = random.randint(self.eat_time+1, NAPPING_STARVING_MAX)
    
    async def set_state(self, new_state):
        async with self.state_condition:
            self.state = new_state
            self.state_condition.notify_all()   # notifies watchers of state changes

# State transition logic for each philosopher
async def transition_philo_state(idx):
    global start, philosophers
    curPhilosopher = philosophers[idx]
    while cond:
        match curPhilosopher.state:
            case PhilosopherState.STARVING:
                # do nothing but wait
                await asyncio.sleep(0.1)
            case PhilosopherState.EATING:
                await asyncio.sleep(curPhilosopher.eat_time)
                curPhilosopher.left_fork.release()
                curPhilosopher.right_fork.release()
                await curPhilosopher.set_state(PhilosopherState.NAPPING)
            case PhilosopherState.NAPPING:
                await asyncio.sleep(curPhilosopher.nap_time)
                await curPhilosopher.set_state(PhilosopherState.STARVING)
        await asyncio.sleep(0.1)    # small delay for CPU cycle fairness


# RGB LED and row LEDs require different value to be set for GREEN value
async def writeGreenLed(idx,val):
    global board_leds, start
    # Setting appropriate 'green' LED value depending on which LED we are toggling
    greenVal = 0    # default off
    if val != 0:
        if (idx < 4):   # Row LEDs
            greenVal = 0x1
        else:           # RGB LED
            greenVal = 0x2
    board_leds[idx].write(greenVal)

# Flashes Philosopher's LED with a period that is controlled by philosopher state
async def flash_led(idx):
    global cond, start, philosophers
    while cond:
        philosopher = philosophers[idx]
        duty_cycle = 0.5    # fixed
        match philosopher.state:
            case PhilosopherState.STARVING:
                period = 0
            case PhilosopherState.EATING:
                period = 0.25
            case PhilosopherState.NAPPING:
                period = 1.5
            case _:
                period = 0
        if period != 0:
            await writeGreenLed(idx,1)
            await asyncio.sleep(period*duty_cycle)
            await writeGreenLed(idx,0)
            await asyncio.sleep(period-period*duty_cycle)
        else:
            await writeGreenLed(idx,0)
            await asyncio.sleep(1)
    await writeGreenLed(idx, 0)


# Push button logic for exiting control loop and terminating program
async def get_btns(_loop):
    global cond, start, philosophers, fork_pickup
    while cond:
        await asyncio.sleep(0.1)
        if (base.buttons[3].read() != 0) and (base.buttons[2].read() != 0):
            _loop.stop()
            cond = False
            await asyncio.gather(*asyncio.all_tasks() - {asyncio.current_task()})

# Philosopher fork pickup logic:
#   - Each philosopher should only pick up a fork if it is in STARVING state
#   - Picking up a fork should be an atomic operation (cannot be pre-empted) to avoid race conditions
async def attempt_fork_pickup(idx):
    global philosophers, fork_pickup
    while cond:
        curPhilosopher = philosophers[idx]
        # waiting for assigned philosopher to be starving before attempting to pickup forks
        async with curPhilosopher.state_condition:
            await curPhilosopher.state_condition.wait_for(lambda: curPhilosopher.state == PhilosopherState.STARVING)
        # print(f"Philosopher {idx} is {curPhilosopher.state.name} and is waiting for its turn to pick up...")
        async with fork_pickup:
            # print(f"Philosopher {idx} is {curPhilosopher.state.name} attempting to pick up the forks...")
            # print(f"Philosopher {idx} left fork locked: {curPhilosopher.left_fork.locked()}")
            # print(f"Philosopher {idx} right fork locked: {curPhilosopher.right_fork.locked()}")
            if (not curPhilosopher.left_fork.locked()) and (not curPhilosopher.right_fork.locked()) \
            and (curPhilosopher.state == PhilosopherState.STARVING):
                curPhilosopher.left_fork.acquire(True)
                curPhilosopher.right_fork.acquire(True)
                await curPhilosopher.set_state(PhilosopherState.EATING)
                # print(f"Philosopher {idx} picked up both forks and is now {curPhilosopher.state.name}...")
        await asyncio.sleep(0.3)
                
# Fork generation
forks = [
    Fork(i)
    for i in range(5)
]

# Philosopher generation
philosophers = [
    Philosopher(
        left_fork=forks[i],   # Left fork is the one at index i
        right_fork=forks[(i + 1) % 5],  # Right fork wraps around using modulo
        led=i
    )
    for i in range(5)
]


In [29]:
# asyncio initialization
loop = asyncio.new_event_loop()
for i in range(5):
    loop.create_task(flash_led(i))
    loop.create_task(transition_philo_state(i))
    loop.create_task(attempt_fork_pickup(i))
loop.create_task(get_btns(loop))
loop.run_forever()

loop.close()  


# Asynchronous Layer
- Button reads
- LED toggles
- LED pulsewidth toggles

# Communication Layer
- Global variables?
- 